import

In [1]:
import matplotlib.pyplot as plt
import torch
import os
import pandas as pd

from tqdm.notebook import tqdm as tqdm
from tqdm.auto import trange

import warnings
warnings.filterwarnings("ignore")

from myhelpers import config_plots, TrialStatistics
from HGNN.train import CNN, dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from flashtorch_modefied import CNN_wrapper, Backprop, SaliencyMap

config_plots.global_settings()

experimetnsFileName = "experiments.csv"

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish"
experimentName="biology_paper_medium_curated3_50_30"
trial_hash="d136f2dc84aaaee7ea2420a87f4ece93e6fb400d60ad702781ed1d11"

# num_workers=8
# batch_size=64

cuda=6

cuda

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 6


importLoad experiment

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

experimentPathAndName = os.path.join(experimentsPath, experimentName)

datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  

load dataset and model

In [5]:
# Get experiment parameters
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
else:
    raise Exception("Experiment not " + trial_hash + " found!")
experimentRecord = experiments_df[experiments_df["trialHash"] == trial_hash]
experiment_params = experimentRecord.to_dict('records')[0]
print(experiment_params)

# Create the model
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
train_loader, validation_loader, test_loader = datasetManager.getLoaders()
fineList = test_loader.dataset.csv_processor.getFineList()
coarseList = test_loader.dataset.csv_processor.getCoarseList()
numberOffine = len(fineList)
numberOfcoarse = len(coarseList)
architecture = {
    "fine": numberOffine,
    "coarse" : numberOfcoarse
}
model = CNN.create_model(architecture, experiment_params, device=cuda)

{'experimentName': 'biology_paper_medium_curated3_50_30', 'modelName': 'models/d136f2dc84aaaee7ea2420a87f4ece93e6fb400d60ad702781ed1d11', 'datasetName': 'datasplits/46951392423537f47b73e60a13110e434f3db01bb4ee6b1ffb48969f', 'experimentHash': '8cfa4d66f84aa8a449c7b08a577cd3cbeaed4558f2f2e77b7edda21c', 'trialHash': 'd136f2dc84aaaee7ea2420a87f4ece93e6fb400d60ad702781ed1d11', 'image_path': 'Curated3/Medium', 'suffix': 'curated_30_50', 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'HGNN_add', 'lambda': 0.01, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9}
Creating datasets...
Creating datasets... Done.
Creating loaders...
Creating loaders... Done.


Load model of a specific trial

In [6]:
# get the model and the parameters
modelName = experimentRecord.iloc[0]["modelName"]
trialName = os.path.join(experimentPathAndName, modelName)

df, epochs, time_elapsed = CNN.loadModel(model, trialName, device=cuda)

Model wrapper

In [7]:
wrapped_model = CNN_wrapper(model, experiment_params, test_loader.dataset)
saliencyMap = SaliencyMap(test_loader.dataset, wrapped_model, experimentPathAndName, trial_hash, experiment_params)

Iterate and give score

In [8]:
from PIL import Image

import gc


def getAverageCorrectProb(loader, layerName, box_width, topk):
    prob = []
    with tqdm(total=len(loader.dataset)) as bar:
        for img in loader:
            fileNames = img['fileNameFull']
            lbls=img[layerName]
            for i, fileName in enumerate(fileNames):
                if topk > 0:
                    _, A = saliencyMap.GetSaliencyMap(fileName,layerName,box_width =box_width, maxCovered=True, topk=topk, plot=False, use_gpu=True,generate_all_steps=False)
                else:
                    isSpecies = (layerName == "fine")
                    activationOutputs = {
                        "fine": isSpecies,
                        "coarse" : not isSpecies
                    }
                    img = saliencyMap.getTransformedImage(Image.open(fileName), False, True)
                    wrapped_model.setOutputsOfInterest(activationOutputs)

                    if torch.cuda.is_available():
                        img = img.cuda()
                    A = wrapped_model(img)
                    
                if (layerName == "coarse" or layerName == "fine"):
                    A = torch.nn.Softmax(dim=1)(A)
                
                if torch.cuda.is_available():
                    A = A.detach().cpu()
                prob_ = A[0][lbls[i]]
                prob.append(prob_)

                bar.update()

    return (sum(prob) / len(prob)).item()


def add_occlusion_result(df, loader, func, label, patchsize, iterations):
    result = func(loader, label, patchsize, iterations)
    return df.append(pd.DataFrame({
        "label": [label],
        "patch size": [patchsize],
        "iterations": [iterations],
        "average correct probability": [result]
    }))

In [9]:
# Create the test loader with small batch
# test_loader = torch.utils.data.DataLoader(test_loader.dataset, batch_size=batch_size, num_workers=num_workers)

df = pd.DataFrame()

In [10]:
df = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", 19, 0)

In [11]:
df = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", 19, 1)

In [12]:
# df = df.append(pd.DataFrame({
#         "label": ["test"],
#         "patch size": [19],
#         "iterations": [1],
#         "average correct probability": [1.23]
#     }))

In [13]:
df = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", 19, 2)

In [14]:
df = add_occlusion_result(df, test_loader, getAverageCorrectProb, "coarse", 19, 0)

In [15]:
df = add_occlusion_result(df, test_loader, getAverageCorrectProb, "coarse", 19, 1)

In [16]:
df = add_occlusion_result(df, test_loader, getAverageCorrectProb, "coarse", 19, 2)

KeyboardInterrupt: 

In [17]:
df = df.reset_index()

In [18]:
df.to_csv(os.path.join(experimentPathAndName, "models", trial_hash, 'occlusion.csv'))

In [19]:
df

,index,label,patch size,iterations,average correct probability
0,0,fine,19,0,0.545186
1,0,fine,19,1,0.469400
2,0,fine,19,2,0.440682
3,0,coarse,19,0,0.171669
4,0,coarse,19,1,0.156280
